Sequential Chats and Customer Onboarding

## Setup

In [1]:
# from utils import get_openai_api_key
# OPENAI_API_KEY = get_openai_api_key()
#print(OPENAI_API_KEY)
#llm_config = {"model": "gpt-3.5-turbo","api_key":OPENAI_API_KEY}

In [2]:
llm_config = {"model": "gpt-oss:latest","api_type":"ollama"}

In [4]:
# STEP 1: Check Ollama server
import requests
try:
    r = requests.get("http://localhost:11434")
    print("Ollama server is running! Status code:", r.status_code)
except Exception as e:
    print("Cannot reach Ollama server:", e)
    exit(1)

Ollama server is running! Status code: 200


In [6]:

# STEP 2: Check model
import subprocess
def is_model_pulled(model_name="gpt-oss:latest"):
    try:
        result = subprocess.run(["ollama", "list"], capture_output=True, text=True, check=True)
        return model_name in result.stdout
    except Exception as e:
        print(f"Error checking model list: {e}")
        return False

if not is_model_pulled("gpt-oss:latest"):
    print("Model is NOT pulled.")
    exit(1)
print("Model is pulled and available.")

Model is pulled and available.


In [7]:
from autogen import ConversableAgent

## Creating the needed agents

In [8]:
from utils import get_openai_api_key
OPENAI_API_KEY = get_openai_api_key()

llm_config = {
    "model": "gpt-oss:latest",
    "base_url": "http://localhost:11434/v1",  # Ollama REST API endpoint
    "api_key": "ollama",  # dummy key (ignored by Ollama but AutoGen requires it)
}

from autogen import ConversableAgent

onboarding_personal_information_agent = ConversableAgent(
    name="onboarding_personal_information_agent",  # Removed spaces
    system_message='''You are a helpful customer onboarding agent.
    You are here to help new customers get started with our product.
    Your job is to gather customer's name and location.
    Do not ask for other information. Return 'TERMINATE' 
    when you have gathered all the information.''',
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode="NEVER",
)

onboarding_topic_preference_agent = ConversableAgent(
    name="onboarding_topic_preference_agent",  # Removed spaces
    system_message='''You are a helpful customer onboarding agent.
    You are here to help new customers get started with our product.
    Your job is to gather customer's preferences on news topics.
    Do not ask for other information.
    Return 'TERMINATE' when you have gathered all the information.''',
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode="NEVER",
)

customer_engagement_agent = ConversableAgent(
    name="customer_engagement_agent",  # Removed spaces
    system_message='''You are a helpful customer service agent.
    You are here to provide fun for the customer based on the user's
    personal information and topic preferences.
    This could include fun facts, jokes, or interesting stories.
    Make sure to make it engaging and fun!
    Return 'TERMINATE' when you are done.''',
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: "terminate" in msg.get("content").lower(),
)

customer_proxy_agent = ConversableAgent(
    name="customer_proxy_agent",
    llm_config=False,
    code_execution_config=False,
    human_input_mode="ALWAYS",
    is_termination_msg=lambda msg: "terminate" in msg.get("content").lower(),
)


## Creating tasks

Now, you can craft a series of tasks to facilitate the onboarding process.

In [9]:
chats = [
    {
        "sender": onboarding_personal_information_agent,
        "recipient": customer_proxy_agent,
        "message": "Hello, I'm here to help you get started with our product. Could you tell me your name and location?",
        "summary_method": "reflection_with_llm",
        "summary_args": {
            "summary_prompt": "Return the customer information as JSON object only: {'name': '', 'location': ''}",
        },
        "max_turns": 1,
        "clear_history": True
    },
    {
        "sender": onboarding_topic_preference_agent,
        "recipient": customer_proxy_agent,
        "message": "Great! Could you tell me what topics you are interested in reading about?",
        "summary_method": "reflection_with_llm",
        "max_turns": 1,
        "clear_history": False
    },
    {
        "sender": customer_proxy_agent,
        "recipient": customer_engagement_agent,
        "message": "Let's find something fun to read.",
        "max_turns": 1,
        "summary_method": "reflection_with_llm",
    },
]

## Start the onboarding process

**Note**: You might get a slightly different response than what's shown in the video. Feel free to try different inputs, such as name, location, and preferences.

In [10]:
from autogen import initiate_chats

chat_results = initiate_chats(chats)
clear_history=False


********************************************************************************
Starting a new chat....

********************************************************************************
onboarding_personal_information_agent (to customer_proxy_agent):

Hello, I'm here to help you get started with our product. Could you tell me your name and location?

--------------------------------------------------------------------------------


C:\Users\User\anaconda3\envs\ckd_env_clean\lib\site-packages\autogen\agentchat\chat.py:57: UserWarning: Repetitive recipients detected: The chat history will be cleared by default if a recipient appears more than once. To retain the chat history, please set 'clear_history=False' in the configuration of the repeating agent.
  warnings.warn(


Replying as customer_proxy_agent. Provide feedback to onboarding_personal_information_agent. Press enter to skip and use auto-reply, or type 'exit' to end the conversation:  Rathiga singapore


customer_proxy_agent (to onboarding_personal_information_agent):

Rathiga singapore

--------------------------------------------------------------------------------

>>>>>>>> TERMINATING RUN (35cbb0e5-b117-4905-8f1a-5a84ace3a37e): Maximum turns (1) reached
[autogen.oai.client: 08-09 12:16:19] {714} WARNING - Model gpt-oss:latest is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]} for customized pricing.

********************************************************************************
Starting a new chat....

********************************************************************************
onboarding_topic_preference_agent (to customer_proxy_agent):

Great! Could you tell me what topics you are interested in reading about?
Context: 
{"name":"Rathiga","location":"Singapore"}

--------------------------------------------------------------------------------


Replying as customer_proxy_agent. Provide feedback to onboarding_topic_preference_agent. Press enter to skip and use auto-reply, or type 'exit' to end the conversation:  Gen AI Agent


customer_proxy_agent (to onboarding_topic_preference_agent):

Gen AI Agent

--------------------------------------------------------------------------------

>>>>>>>> TERMINATING RUN (c2200340-63e4-4a67-b0c8-c0c8ec676c5a): Maximum turns (1) reached
[autogen.oai.client: 08-09 12:23:14] {714} WARNING - Model gpt-oss:latest is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]} for customized pricing.

********************************************************************************
Starting a new chat....

********************************************************************************
customer_proxy_agent (to customer_engagement_agent):

Let's find something fun to read.
Context: 
{"name":"Rathiga","location":"Singapore"}
I’m particularly drawn to content that spans a mix of technology, science, and culture. Here are a few topics I love exploring:

1. **Artificial Intelligence & Machine Learning** – From cutting‑e

## Print out the summary

In [13]:
summary=[]
for chat_result in chat_results:
    print(chat_result.summary)
    summary.append(chat_result.summary)
    print("\n")

{"name":"Rathiga","location":"Singapore"}


I’m particularly drawn to content that spans a mix of technology, science, and culture. Here are a few topics I love exploring:

1. **Artificial Intelligence & Machine Learning** – From cutting‑edge research to real‑world applications.  
2. **Emerging Tech & Gadgets** – Smart cities, wearables, sustainable tech, and quantum computing.  
3. **Data Science & Analytics** – Storytelling with data, predictive modeling, and ethical AI concerns.  
4. **Health & Wellness Innovation** – Telemedicine, personalized nutrition, and mental health tech.  
5. **Sustainability & Climate Solutions** – Circular economy, green infrastructure, and impact investing.  
6. **Space Exploration & Astronomy** – Mars missions, exoplanet studies, and the future of space travel.  
7. **Cultural Perspectives** – Global arts, film reviews, and the evolving narratives of city life in places like Singapore.  
8. **Education & Skill‑building** – Online learning platforms, codi

In [14]:
summary[0]

'{"name":"Rathiga","location":"Singapore"}'

In [9]:
#!pip install pandas

In [15]:
import pandas as pd
import ast

# Input string
#data_str = "{'name': 'Ramisha', 'location': 'Coimbatore'}"
data_str=summary[0]
# Convert string to dictionary
data_dict = ast.literal_eval(data_str)

# Convert dictionary to DataFrame
df = pd.DataFrame([data_dict])

df


,name,location
0,Rathiga,Singapore


In [16]:
df["TopicPreference"]=summary[1]

In [17]:
df

,name,location,TopicPreference
0,Rathiga,Singapore,I’m particularly drawn to content that spans a...


In [18]:
df["Enagement"]=summary[2]

In [19]:
df

,name,location,TopicPreference,Enagement
0,Rathiga,Singapore,I’m particularly drawn to content that spans a...,"User seeks engaging reads that blend AI, emerg..."


In [50]:
# !pip install gspread
# !pip install oauth2client
# !pip install gspread_formatting

In [55]:
def add_leads_to_google_sheet(df):
    import gspread
    from oauth2client.service_account import ServiceAccountCredentials
    from gspread_formatting import format_cell_range, CellFormat, TextFormat


    # Define the scope for Google Sheets API
    scope = ["https://www.googleapis.com/auth/spreadsheets","https://www.googleapis.com/auth/drive"]
    # Add credentials to the account
    creds = ServiceAccountCredentials.from_json_keyfile_name(
        'firm-circlet-468504-h0-157edabda0a2.json', scope
    )

    # Authorize the client
    client = gspread.authorize(creds)

    # Specify the Google Sheet ID
    sheet_id = '1h47faix8YqGKq-9uxZ_3daWczMpKH-bQnQDAMcV0pNo'

    # Open the spreadsheet using its ID
    spreadsheet = client.open_by_key(sheet_id)

    # Select the first worksheet (index 0)
    worksheet = spreadsheet.get_worksheet(0)

    # Append each row of the DataFrame to the worksheet
    for _, row in df.iterrows():
        # Convert row to a list and append to the sheet
        worksheet.append_row(row.tolist())
    # Apply wrap text formatting to all cells
    fmt = CellFormat(wrapStrategy='WRAP')
    format_cell_range(worksheet, 'A:Z', fmt)  # A:Z = first 26 columns

    print("Data added successfully!")
    return "added"


In [ ]:
def add_leads_to_google_sheet(df):
    import gspread
    from oauth2client.service_account import ServiceAccountCredentials
    from gspread_formatting import format_cell_range, CellFormat, TextFormat


    # Define the scope for Google Sheets API
    scope = ["https://www.googleapis.com/auth/spreadsheets","https://www.googleapis.com/auth/drive"]
    # Add credentials to the account
    creds = ServiceAccountCredentials.from_json_keyfile_name(
        '<your json>', scope
    )

    # Authorize the client
    client = gspread.authorize(creds)

    # Specify the Google Sheet ID
    sheet_id = '<sheet ID>'

    # Open the spreadsheet using its ID
    spreadsheet = client.open_by_key(sheet_id)

    # Select the first worksheet (index 0)
    worksheet = spreadsheet.get_worksheet(0)

    # Append each row of the DataFrame to the worksheet
    for _, row in df.iterrows():
        # Convert row to a list and append to the sheet
        worksheet.append_row(row.tolist())
    # Apply wrap text formatting to all cells
    fmt = CellFormat(wrapStrategy='WRAP')
    format_cell_range(worksheet, 'A:Z', fmt)  # A:Z = first 26 columns

    print("Data added successfully!")
    return "added"


In [56]:
add_leads_to_google_sheet(df)

Data added successfully!


'added'

## Print out the cost

In [57]:
for chat_result in chat_results:
    print(chat_result.cost)
    print("\n")

{'usage_including_cached_inference': {'total_cost': 0, 'gpt-oss:latest': {'cost': 0, 'prompt_tokens': 125, 'completion_tokens': 251, 'total_tokens': 376}}, 'usage_excluding_cached_inference': {'total_cost': 0, 'gpt-oss:latest': {'cost': 0, 'prompt_tokens': 125, 'completion_tokens': 251, 'total_tokens': 376}}}


{'usage_including_cached_inference': {'total_cost': 0, 'gpt-oss:latest': {'cost': 0, 'prompt_tokens': 129, 'completion_tokens': 480, 'total_tokens': 609}}, 'usage_excluding_cached_inference': {'total_cost': 0, 'gpt-oss:latest': {'cost': 0, 'prompt_tokens': 129, 'completion_tokens': 480, 'total_tokens': 609}}}


{'usage_including_cached_inference': {'total_cost': 0, 'gpt-oss:latest': {'cost': 0, 'prompt_tokens': 1146, 'completion_tokens': 801, 'total_tokens': 1947}}, 'usage_excluding_cached_inference': {'total_cost': 0, 'gpt-oss:latest': {'cost': 0, 'prompt_tokens': 1146, 'completion_tokens': 801, 'total_tokens': 1947}}}


